In [1]:
import torch
import pandas as pd
import numpy as np
import pytorch_transformers

folder = 'd:\git\project1\data'

In [2]:
model_class = pytorch_transformers.BertModel
tokenizer_class = pytorch_transformers.BertTokenizer
pretrained_weights = 'bert-base-uncased'

model = model_class.from_pretrained(pretrained_weights)

In [3]:
%%time
tokens_padded = pd.read_csv(f'{folder}/retweets_padded1.csv')
tokens_padded

Wall time: 910 ms


,retweets,0,1,2,3,4,5,6,7,8,...,40,41,42,43,44,45,46,47,48,49
0,0,101,1041,10439,8189,15529,9956,4895,16371,16059,...,2015,102,0,0,0,0,0,0,0,0
1,1,101,2178,3231,1056,28394,2102,2008,2347,1005,...,0,0,0,0,0,0,0,0,0,0
2,0,101,1030,22064,17139,23460,2057,2031,2042,2311,...,0,0,0,0,0,0,0,0,0,0
3,0,101,1030,7707,2532,12722,18609,15697,13820,2099,...,0,0,0,0,0,0,0,0,0,0
4,1,101,11241,2026,2166,10995,2046,2978,3597,2378,...,1013,29347,2102,16048,2102,2620,2050,26976,102,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47277,0,101,1030,19888,24163,29148,2129,2152,2978,3597,...,0,0,0,0,0,0,0,0,0,0
47278,0,101,1030,1996,2389,5558,1030,3682,2595,2099,...,0,0,0,0,0,0,0,0,0,0
47279,0,101,2196,1010,2412,3543,2978,3597,2378,10518,...,1013,1050,4160,2575,8523,4160,5358,2497,102,0
47280,1,101,100,8866,1001,2978,3597,2378,16770,1024,...,0,0,0,0,0,0,0,0,0,0


In [4]:
%%time
padded = np.array(tokens_padded.iloc[:,1:])
attention_mask = np.where(padded != 0, 1, 0)
input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

input_ids

Wall time: 134 ms


tensor([[  101,  1041, 10439,  ...,     0,     0,     0],
        [  101,  2178,  3231,  ...,     0,     0,     0],
        [  101,  1030, 22064,  ...,     0,     0,     0],
        ...,
        [  101,  2196,  1010,  ...,  2497,   102,     0],
        [  101,   100,  8866,  ...,     0,     0,     0],
        [  101,  1060, 14536,  ...,     0,     0,     0]])

In [28]:
def save_last_hidden(start_index, end_index):
    last_hidden_states = model(input_ids[start_index:end_index, :], attention_mask=attention_mask[start_index:end_index, :])
    last_hidden_states_df = pd.DataFrame(last_hidden_states[0][:,0,:].numpy())
    #print(np.array(tokens_padded.loc[start_index:end_index-1, ['retweets']]))
    last_hidden_states_df['retweet'] = np.array(tokens_padded.loc[start_index:end_index-1, ['retweets']])
    #print(last_hidden_states_df)
    last_hidden_states_df.to_csv(f'{folder}/last_hidden_states_{start_index}.csv', index=False)

In [30]:
%%time
rows_count = input_ids.shape[0]
chank_size = 1000
start = 2
for chank_num in range (start, (int)(rows_count/chank_size)+1):
    print(chank_num)
    with torch.no_grad():
        start_index = chank_num*chank_size
        end_index = (chank_num+1)*chank_size
        if (end_index > rows_count):
            end_index=rows_count-1
        print(start_index)
        print(end_index)
        save_last_hidden(start_index, end_index)
        

2
2000
3000
3
3000
4000
4
4000
5000
5
5000
6000
6
6000
7000
7
7000
8000
8
8000
9000
9
9000
10000
10
10000
11000
11
11000
12000
12
12000
13000
13
13000
14000
14
14000
15000


KeyboardInterrupt: 

In [29]:
%%time
with torch.no_grad():
    save_last_hidden(1000, 2000)

Wall time: 3min 17s
